# **IP1_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 35.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsuitesparseconfig5 amd64 1:5.10.1+dfsg-4build1 [10.4 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libamd2 amd64 1:5.10.1+dfsg-4build1 [21.6 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 libcolamd2 amd64 1:5.10.1+dfsg-4build1 [18.0 k

In [ ]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better.

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions.

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i.  """


In [ ]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Here, we define our variables.

- Let Xij be the binary decision variable that is equal to 1 if patient i is assigned j proton fractions and 0 otherwise.

- Let Ci be the number of proton fractions patient i receives.

We are formulating these variables in binary integer form since the decision to assign certain proton fractions to a patient can either be made (equals 1) or not (equals 0).

We use Ci to represent the exact number of proton fractions a patient receives, this is a continuous variable where 0 <= Ci <= 15, assuming a patient cannot receive more than 15 fractions. 

More specifically,
- i ∈ {1,2,...,17} representing each patient
- j ∈ {0,1,...,15} representing the number of proton fractions  

We are assuming that the BED scores for each possible number of proton fractions for each patient (BEDi(j,15-j)) are known beforehand and available from the data provided.


In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

The objective function can be formulated as follows:

Maximize Σ (BEDi(j,15-j) * Xij), for i = 1 to 17, j = 0 to 15. 

This objective function is subject to the constraint that the sum of proton fractions for all patients (Ci) should not exceed the total capacity C, i.e., Σ Ci ≤ C, for i = 1 to 17.

Please note: The full integer linear optimization model would also need to include further constraints specifying the binary nature of the Xij variables (i.e., Xij={0,1}), and the range of the Ci variables (0 <= Ci <= 15). But these are not part of the objective function.

In [ ]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response3.choices[0].message.content)

The constraints for this optimization problem are:

1. Each patient i should receive exactly 15 fractions in total, whether they are photon or proton fractions. Therefore, the sum of photon and proton fractions for each patient should be exactly 15:

   Σ Xij * j for all j in {0,...,15} = 15, for all i in {1,...,17}

2. The total number of proton fractions used across all patients i should not exceed the total capacity C, i.e., Σ Ci ≤ C, for all i in {1,...,17}.

3. The number of proton fractions received by each patient i equals the j value when Xij = 1. Therefore, the sum of j * Xij for all j in {0,...,15} should equal Ci. 

   Σ j * Xij for all j in {0,...,15} = Ci , for all i in {1,...,17} 

4. The binary variables Xij can either be 0 or 1:

   Xij <= 1 and Xij >= 0, ∀i in {1,...,17}, ∀j in {0,1,...,15} and Xij = {0,1} integer

5. The fractions Ci each patient i receives should be within the range 0 and 15:

   0 <= Ci <= 15, ∀i in {1,...,17} and Ci is a real number. 

These constraints ensure that each patient receives exactly the suitable therapy fractions they need, the total therapy fraction usage does not exceed the available capacity, and each therapy fraction decision is binary (i.e., the fraction is either used or not used). The last two constraints ensure respective mathematical structures, with Xij being binary and Ci being real numbers ranging from 0 to 15. 

done 


### **Generate Pyomo Code**

In [ ]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response4.choices[0].message.content)

Here's how the problem can be translated to Pyomo code:

```python
from pyomo.environ import *

# Sample data
n_patients = 17
n_fractions = 15
total_capacity = 200

# BED scores for each patient and each proton fraction
# We are assuming a random BED score for showcasing the Pyomo code
import random
BED_score = {i: {j: random.randint(80, 150) for j in range(n_fractions + 1)} for i in range(n_patients)}

# Model
model = ConcreteModel()

# Sets
model.I = RangeSet(n_patients)
model.J = RangeSet(n_fractions)

# Variables
model.X = Var(model.I, model.J, within=Binary) 
model.C = Var(model.I, within=NonNegativeReals, bounds=(0, n_fractions))

# Objective
model.obj = Objective(expr = sum(BED_score[i][j] * model.X[i,j] for i in model.I for j in model.J), sense=maximize)

# Constraints
model.frac_cons = ConstraintList()
for i in model.I:
    model.frac_cons.add(sum(j * model.X[i,j] for j in model.J) == n_fractions)  # Constraint 1
    model.frac_cons.add(sum(j * model.X[i,j] for j in model.J) == model.C[i])   # Constraint 3

model.total_capacity_cons = Constraint(expr = sum(model.C[i] for i in model.I) <= total_capacity)  # Constraint 2

SolverFactory('glpk').solve(model)

model.pprint()
```

Here we use Binary variable type for Xij and NonNegativeReals for Ci. The obj function seeks to maximize the sum of BED scores across all i and j. The constraints ensure the total fraction for each patient is 15, the fractions agree with the decision variables, and the total therapy fraction usage does not exceed the available capacity. Note also that GLPK solver is used here. 

Note: Please install GLPK solver and Pyomo Library in Python by using pip command, if you haven't done so:

```python
pip install pyomo
apt-get install -y -qq glpk-utils
done 
```

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# Sample data
n_patients = 17
n_fractions = 15
total_capacity = 100

# BED scores for each patient and each proton fraction
# We are assuming a random BED score for showcasing the Pyomo code
import random
BED_score = {i: {j: random.randint(80, 150) for j in range(n_fractions + 1)} for i in range(n_patients)}

# Model
model = ConcreteModel()

# Sets
model.I = RangeSet(n_patients)
model.J = RangeSet(n_fractions)

# Variables
model.X = Var(model.I, model.J, within=Binary)
model.C = Var(model.I, within=NonNegativeReals, bounds=(0, n_fractions))

# Objective
model.obj = Objective(expr = sum(BED_score[i][j] * model.X[i,j] for i in model.I for j in model.J), sense=maximize)

# Constraints
model.frac_cons = ConstraintList()
for i in model.I:
    model.frac_cons.add(sum(j * model.X[i,j] for j in model.J) == n_fractions)  # Constraint 1
    model.frac_cons.add(sum(j * model.X[i,j] for j in model.J) == model.C[i])   # Constraint 3

model.total_capacity_cons = Constraint(expr = sum(model.C[i] for i in model.I) <= total_capacity)  # Constraint 2

SolverFactory('glpk').solve(model)

KeyError: 17

In [ ]:
from pyomo.environ import *
import pandas as pd
# Sample data
n_patients = 16
n_fractions = 15
total_capacity = 100

# BED scores for each patient and each proton fraction
# We are assuming a random BED score for showcasing the Pyomo code

#LOADING THE DATA DONE BY HUMAN
df = pd.read_csv('ProblemData.csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j)] = dose
#End


# Model
model = ConcreteModel()

# Sets
model.I = RangeSet(n_patients)
model.J = RangeSet(n_fractions)

# Variables
model.X = Var(model.I, model.J, within=Binary)
model.C = Var(model.I, within=NonNegativeReals, bounds=(0, n_fractions))

# Objective
model.obj = Objective(expr = sum(BED_score[i][j] * model.X[i,j] for i in model.I for j in model.J), sense=maximize)

# Constraints
model.frac_cons = ConstraintList()
for i in model.I:
    model.frac_cons.add(sum(j * model.X[i,j] for j in model.J) == n_fractions)  # Constraint 1
    model.frac_cons.add(sum(j * model.X[i,j] for j in model.J) == model.C[i])   # Constraint 3

model.total_capacity_cons = Constraint(expr = sum(model.C[i] for i in model.I) <= total_capacity)  # Constraint 2

SolverFactory('glpk').solve(model)

{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 33, 'Number of variables': 256, 'Number of nonzeros': 512, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'infeasible', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.04293465614318848}]}

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**